In [2]:
import pandas as pd
import numpy as np
import os
import dotenv
from dotenv import load_dotenv
from common.PortConnect import Port_Connect

In [ ]:
load_dotenv()
api_key = os.getenv("API_KEY")

fmp = Port_Connect(api_key=api_key)
apple = fmp.historical_closing_price('BRK.B')
apple



/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)


,symbol,close
date,,
1996-05-09,BRK.B,23.20
1996-05-10,BRK.B,24.00
1996-05-13,BRK.B,23.90
1996-05-14,BRK.B,23.60
1996-05-15,BRK.B,23.20
...,...,...
2024-08-22,BRK.B,449.03
2024-08-23,BRK.B,453.38
2024-08-26,BRK.B,454.49


## Calculate Key Metric Statistics for a holding

In [ ]:
from common.Portfolio import Portfolio_Stats








In [22]:
port = Portfolio_Stats()

In [23]:
change = apple['close'].pct_change()
change

date
1996-05-09         NaN
1996-05-10    0.034483
1996-05-13   -0.004167
1996-05-14   -0.012552
1996-05-15   -0.016949
                ...   
2024-08-22    0.005441
2024-08-23    0.009688
2024-08-26    0.002448
2024-08-27    0.013510
2024-08-28    0.008597
Name: close, Length: 7124, dtype: float64

In [ ]:
type(change)dedededed

pandas.core.series.Series

In [ ]:
filtered_change = change.loc['2010-01-01':'2025-12-31'

In [26]:
port.annualize_rets(filtered_change,252)

0.14297668079959802

In [27]:
port.annualize_vol(filtered_change,252)

0.19516056867051634

In [ ]:
max_duration = port.drawdown_duration_from_returns(filtered_change)
max_duration

692

In [29]:
fmp.get_closing_prices('BRK.B')

/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:250: UserWarning: 

symbol,BRK.B
date,
1996-05-09,23.20
1996-05-10,24.00
1996-05-13,23.90
1996-05-14,23.60
1996-05-15,23.20
...,...
2024-08-22,449.03
2024-08-23,453.38
2024-08-26,454.49


In [40]:
# tickers = ['SYK','ISRG','BLK','BSX','HD',"SPY","BRK.B","INDA","BBCA","EWJ","EWA","EWL","EWQ","KWEB","EWZ","EWY"]

tickers = [
    "SPY",
    "ARGT",
    "EWA",
    "EWO",
    "EWK",
    "EWZ",
    "EWC",
    "ECH",
    "MCHI",
    "GXG",
    "EDEN",
    "EGPT",
    "EFNL",
    "EWQ",
    "EWG",
    "GREK",
    "EWH",
    "INDA",
    "EIDO",
    "EIRL",
    "EIS",
    "EWI",
    "EWJ",
    "KWT",
    "EWM",
    "EWW",
    "EWN",
    "ENZL",
    "NGE",
    "NORW",
    "PAK",
    "EPU",
    "EPHE",
    "EPOL",
    "QAT",
    "KSA",
    "EWS",
    "EZA",
    "EWY",
    "EWP",
    "EWD",
    "EWL",
    "EWT",
    "THD",
    "TUR",
    "UAE",
    "EWU",
    "VNM"
]

prices = fmp.get_closing_prices(tickers,from_date='2010-01-01')
prices

/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(df['date'],infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:62: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.set_index = pd.to_datetime(df.index, infer_datetime_format=True)
/Users/franciscoruiz/Desktop/APPLIED_FINANCE_ANALYTICS/APPLIED_FINANCE_ANALYTICS/common/PortConnect.py:56: UserWarning: T

symbol,ARGT,ECH,EDEN,EFNL,EGPT,EIDO,EIRL,EIS,ENZL,EPHE,...,MCHI,NGE,NORW,PAK,QAT,SPY,THD,TUR,UAE,VNM
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,56.10,NaN,NaN,NaN,NaN,NaN,56.56,NaN,NaN,...,NaN,NaN,16.72,NaN,NaN,113.33,43.17,56.10,NaN,27.00
2010-01-05,NaN,56.33,NaN,NaN,NaN,NaN,NaN,56.82,NaN,NaN,...,NaN,NaN,16.51,NaN,NaN,113.63,43.09,56.80,NaN,27.60
2010-01-06,NaN,57.24,NaN,NaN,NaN,NaN,NaN,57.18,NaN,NaN,...,NaN,NaN,16.90,NaN,NaN,113.71,43.17,57.41,NaN,27.43
2010-01-07,NaN,58.58,NaN,NaN,NaN,NaN,NaN,57.00,NaN,NaN,...,NaN,NaN,17.01,NaN,NaN,114.19,43.41,58.45,NaN,27.40
2010-01-08,NaN,59.38,NaN,NaN,NaN,NaN,NaN,57.88,NaN,NaN,...,NaN,NaN,17.10,NaN,NaN,114.57,43.69,58.50,NaN,27.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-16,86.95,31.11,118.19,40.05,NaN,18.01,66.39,86.70,45.97,27.35,...,55.01,NaN,30.73,NaN,18.05,602.68,51.58,30.63,17.56,13.26
2025-06-17,85.90,30.45,114.71,39.35,NaN,18.02,64.88,86.47,45.09,26.98,...,54.44,NaN,30.57,NaN,17.90,597.53,51.02,30.21,17.24,13.31
2025-06-18,86.17,30.34,114.41,39.42,NaN,17.87,65.31,87.29,45.08,26.70,...,53.93,NaN,30.44,NaN,17.82,597.44,49.89,30.11,16.99,13.36


In [41]:
dfs = []

market_index = prices["SPY"].pct_change()

for ticker in prices.columns:
    change = prices[ticker].pct_change()
    summary = port.summary_stats(change,market_index=market_index)
    dfs.append(summary)


pd.concat(dfs).sort_values(by="Annualized Return",ascending=False)


/var/folders/01/l5vdw9pj3n7b0sg_9lmyyzbh0000gn/T/ipykernel_31675/687853882.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  change = prices[ticker].pct_change()
/var/folders/01/l5vdw9pj3n7b0sg_9lmyyzbh0000gn/T/ipykernel_31675/687853882.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  change = prices[ticker].pct_change()
/var/folders/01/l5vdw9pj3n7b0sg_9lmyyzbh0000gn/T/ipykernel_31675/687853882.py:6: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or 

,Annualized Return,Annualized Vol,Annualized Semideviation,Sharpe Ratio,Skewness,Kurtosis,Cornish-Fisher VaR (5%),Historic CVaR (5%),Max Drawdown,Drawdown Duration (Days),Up Days %,Ulcer index,Calmar Ratio,Sortino Ratio,Beta,Correlation
SPY,0.113998,0.174461,0.141299,0.577267,-0.322688,14.811983,0.015953,0.026660,-0.341047,745,0.549100,6.946824,-0.334257,0.663152,1.00,1.00
EDEN,0.098072,0.187809,0.136752,0.514242,-0.471247,7.026903,0.019532,0.027056,-0.375127,895,0.521958,10.398724,-0.261437,0.641625,0.72,0.65
EIRL,0.074974,0.224944,0.176023,0.251990,-0.537737,11.535345,0.022559,0.033563,-0.483117,1044,0.520505,13.470461,-0.155187,0.412653,0.88,0.68
ARGT,0.068193,0.292790,0.221237,0.178084,-0.790747,15.354256,0.029209,0.041936,-0.625944,2358,0.510425,28.909006,-0.108944,0.371909,1.03,0.61
EWN,0.061373,0.222529,0.168492,0.180828,-0.249617,8.871316,0.022037,0.033287,-0.444775,1368,0.530334,14.885596,-0.137987,0.358060,1.04,0.82
EWL,0.056953,0.174717,0.135085,0.193780,-0.504293,11.783259,0.017395,0.025949,-0.303848,1163,0.518509,11.030518,-0.187439,0.383067,0.75,0.75
KWT,0.053127,3.657214,0.288187,0.077878,61.687235,3833.486460,-37.934817,0.047445,-0.864140,4127,0.396401,52.879102,-0.061480,2.288558,0.77,0.04
EWT,0.048189,0.215898,0.167486,0.105477,-0.872512,12.951013,0.022535,0.031699,-0.412719,1266,0.520823,16.960226,-0.116759,0.302653,0.86,0.69
INDA,0.047009,0.223872,0.174656,0.117895,-0.632428,16.282494,0.021532,0.032584,-0.461377,1046,0.511593,12.938009,-0.101888,0.332250,0.80,0.61
EWD,0.039084,0.257173,0.194264,0.045424,-0.355443,8.652182,0.026110,0.038660,-0.460620,2361,0.515681,18.637638,-0.084851,0.265079,1.14,0.78


In [32]:
market_index

date
2010-01-04         NaN
2010-01-05    0.002647
2010-01-06    0.000704
2010-01-07    0.004221
2010-01-08    0.003328
                ...   
2025-06-16    0.009514
2025-06-17   -0.008545
2025-06-18   -0.000151
2025-06-20   -0.005289
2025-06-23    0.009877
Name: SPY, Length: 3891, dtype: float64